# 🔍 Hyperlane Token Fee Explorer

Interactive visualization of Linear, Progressive, and Regressive fee structures.

Use the sliders below to explore how different parameters affect fee calculations.

**Key Transfer Amounts Analyzed:** $100, $1,000, $10,000, $100,000 (USDC: 1 USD = 10^6 tokens)

In [ ]:
# Install required packages (run this cell first if needed)
# !pip install ipywidgets matplotlib numpy

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider, Dropdown, Layout, HBox, VBox, Button, Output
from IPython.display import display, HTML, clear_output

# Enable interactive plots
%matplotlib inline

## Fee Calculation Functions

In [ ]:
def linear_fee(amount, max_fee, half_amount):
    """Linear Fee: fee = min(maxFee, (amount * maxFee) / (2 * halfAmount))"""
    uncapped = (amount * max_fee) / (2 * half_amount)
    return np.minimum(uncapped, max_fee)

def progressive_fee(amount, max_fee, half_amount):
    """Progressive Fee: fee = (maxFee * amount²) / (halfAmount² + amount²)"""
    if isinstance(amount, np.ndarray):
        return np.where(amount == 0, 0,
                       (max_fee * amount**2) / (half_amount**2 + amount**2))
    return 0 if amount == 0 else (max_fee * amount**2) / (half_amount**2 + amount**2)

def regressive_fee(amount, max_fee, half_amount):
    """Regressive Fee: fee = (maxFee * amount) / (halfAmount + amount)"""
    denominator = half_amount + amount
    return np.where(denominator == 0, 0, (max_fee * amount) / denominator)

## Interactive Fee Explorer

**Adjust the sliders below to explore different configurations:**

In [ ]:
# Key transfer amounts to highlight (USDC: 1 USD = 10^6 tokens)
KEY_AMOUNTS = [100 * 10**6, 1000 * 10**6, 10000 * 10**6, 100000 * 10**6]  # $100, $1K, $10K, $100K
KEY_AMOUNT_LABELS = ['$100', '$1K', '$10K', '$100K']
KEY_AMOUNT_COLORS = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#FFA07A']

def plot_fees(max_fee=1000000, half_amount=10000000, selected_amount=10000000):
    """
    Interactive fee visualization with adjustable parameters and key amount markers.
    """
    # Determine appropriate max_amount for display
    max_display = max(100000 * 10**6, max(KEY_AMOUNTS) * 1.2, half_amount * 2)
    
    # Generate amount range
    amounts = np.linspace(0, max_display, 1000)
    
    # Calculate fees
    linear_fees = linear_fee(amounts, max_fee, half_amount)
    progressive_fees = progressive_fee(amounts, max_fee, half_amount)
    regressive_fees = regressive_fee(amounts, max_fee, half_amount)
    
    # Calculate percentages
    with np.errstate(divide='ignore', invalid='ignore'):
        linear_pct = np.where(amounts > 0, (linear_fees / amounts) * 100, 0)
        progressive_pct = np.where(amounts > 0, (progressive_fees / amounts) * 100, 0)
        regressive_pct = np.where(amounts > 0, (regressive_fees / amounts) * 100, 0)
    
    # Create figure with subplots
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    fig.suptitle('Hyperlane Token Fee Structures', fontsize=16, fontweight='bold')
    
    colors = {'linear': '#2E86AB', 'progressive': '#A23B72', 'regressive': '#F18F01'}
    
    # Plot 1: Absolute Fees
    ax1 = axes[0]
    ax1.plot(amounts, linear_fees, label='Linear', linewidth=2.5, color=colors['linear'])
    ax1.plot(amounts, progressive_fees, label='Progressive', linewidth=2.5, color=colors['progressive'])
    ax1.plot(amounts, regressive_fees, label='Regressive', linewidth=2.5, color=colors['regressive'])
    
    # Highlight key transfer amounts
    for i, amt in enumerate(KEY_AMOUNTS):
        if amt <= max_display:
            ax1.axvline(x=amt, color=KEY_AMOUNT_COLORS[i], linestyle=':', alpha=0.4, linewidth=2)
            
            # Add markers at intersection points
            lin_fee = linear_fee(amt, max_fee, half_amount)
            prog_fee = progressive_fee(amt, max_fee, half_amount)
            reg_fee = regressive_fee(amt, max_fee, half_amount)
            
            ax1.scatter([amt], [lin_fee], color=colors['linear'], s=80, zorder=5, 
                       edgecolors=KEY_AMOUNT_COLORS[i], linewidths=2, alpha=0.7)
            ax1.scatter([amt], [prog_fee], color=colors['progressive'], s=80, zorder=5,
                       edgecolors=KEY_AMOUNT_COLORS[i], linewidths=2, alpha=0.7)
            ax1.scatter([amt], [reg_fee], color=colors['regressive'], s=80, zorder=5,
                       edgecolors=KEY_AMOUNT_COLORS[i], linewidths=2, alpha=0.7)
    
    # Highlight selected amount
    if selected_amount > 0:
        current_linear = linear_fee(selected_amount, max_fee, half_amount)
        current_progressive = progressive_fee(selected_amount, max_fee, half_amount)
        current_regressive = regressive_fee(selected_amount, max_fee, half_amount)
        
        ax1.axvline(x=selected_amount, color='red', linestyle='--', alpha=0.6, linewidth=2.5, 
                   label=f'Selected: ${selected_amount/10**6:,.0f}')
        ax1.scatter([selected_amount], [current_linear], color=colors['linear'], s=200, 
                   zorder=6, marker='*', edgecolors='red', linewidths=2)
        ax1.scatter([selected_amount], [current_progressive], color=colors['progressive'], s=200,
                   zorder=6, marker='*', edgecolors='red', linewidths=2)
        ax1.scatter([selected_amount], [current_regressive], color=colors['regressive'], s=200,
                   zorder=6, marker='*', edgecolors='red', linewidths=2)
    
    ax1.axhline(y=max_fee, color='gray', linestyle='--', alpha=0.3, linewidth=1.5, label=f'maxFee')
    ax1.axvline(x=half_amount, color='gray', linestyle='-.', alpha=0.4, linewidth=1.5, label='halfAmount')
    
    ax1.set_xlabel('Transfer Amount (USDC)', fontsize=12, fontweight='bold')
    ax1.set_ylabel('Absolute Fee (USDC)', fontsize=12, fontweight='bold')
    ax1.set_title('Absolute Fees', fontsize=13, fontweight='bold')
    ax1.legend(loc='lower right', fontsize=9)
    ax1.grid(True, alpha=0.3, linestyle=':')
    ax1.set_xlim(0, max_display)
    ax1.set_ylim(0, max_fee * 1.1)
    
    # Format x-axis to show in dollars
    ax1.ticklabel_format(style='plain', axis='x')
    
    # Plot 2: Fee Percentages
    ax2 = axes[1]
    ax2.plot(amounts, linear_pct, label='Linear', linewidth=2.5, color=colors['linear'])
    ax2.plot(amounts, progressive_pct, label='Progressive', linewidth=2.5, color=colors['progressive'])
    ax2.plot(amounts, regressive_pct, label='Regressive', linewidth=2.5, color=colors['regressive'])
    
    # Highlight key transfer amounts
    for i, amt in enumerate(KEY_AMOUNTS):
        if amt <= max_display and amt > 0:
            ax2.axvline(x=amt, color=KEY_AMOUNT_COLORS[i], linestyle=':', alpha=0.4, linewidth=2)
            
            lin_fee = linear_fee(amt, max_fee, half_amount)
            prog_fee = progressive_fee(amt, max_fee, half_amount)
            reg_fee = regressive_fee(amt, max_fee, half_amount)
            
            ax2.scatter([amt], [lin_fee/amt*100], color=colors['linear'], s=80, zorder=5,
                       edgecolors=KEY_AMOUNT_COLORS[i], linewidths=2, alpha=0.7)
            ax2.scatter([amt], [prog_fee/amt*100], color=colors['progressive'], s=80, zorder=5,
                       edgecolors=KEY_AMOUNT_COLORS[i], linewidths=2, alpha=0.7)
            ax2.scatter([amt], [reg_fee/amt*100], color=colors['regressive'], s=80, zorder=5,
                       edgecolors=KEY_AMOUNT_COLORS[i], linewidths=2, alpha=0.7)
    
    # Highlight selected amount
    if selected_amount > 0:
        ax2.axvline(x=selected_amount, color='red', linestyle='--', alpha=0.6, linewidth=2.5,
                   label=f'Selected: ${selected_amount/10**6:,.0f}')
        ax2.scatter([selected_amount], [current_linear/selected_amount*100], color=colors['linear'], 
                   s=200, zorder=6, marker='*', edgecolors='red', linewidths=2)
        ax2.scatter([selected_amount], [current_progressive/selected_amount*100], color=colors['progressive'],
                   s=200, zorder=6, marker='*', edgecolors='red', linewidths=2)
        ax2.scatter([selected_amount], [current_regressive/selected_amount*100], color=colors['regressive'],
                   s=200, zorder=6, marker='*', edgecolors='red', linewidths=2)
    
    ax2.axvline(x=half_amount, color='gray', linestyle='-.', alpha=0.4, linewidth=1.5, label='halfAmount')
    
    ax2.set_xlabel('Transfer Amount (USDC)', fontsize=12, fontweight='bold')
    ax2.set_ylabel('Fee Percentage (%)', fontsize=12, fontweight='bold')
    ax2.set_title('Fee Percentages', fontsize=13, fontweight='bold')
    ax2.legend(loc='best', fontsize=9)
    ax2.grid(True, alpha=0.3, linestyle=':')
    ax2.set_xlim(0, max_display)
    
    # Dynamic ylim for percentage
    max_pct = max(
        np.max(linear_pct[amounts <= half_amount*2]) if len(linear_pct[amounts <= half_amount*2]) > 0 else 10,
        np.max(progressive_pct[amounts <= half_amount*2]) if len(progressive_pct[amounts <= half_amount*2]) > 0 else 10,
        np.max(regressive_pct[amounts <= half_amount*2]) if len(regressive_pct[amounts <= half_amount*2]) > 0 else 10
    )
    ax2.set_ylim(0, min(25, max_pct * 1.3))
    
    # Format x-axis to show in dollars
    ax2.ticklabel_format(style='plain', axis='x')
    
    plt.tight_layout()
    plt.show()
    
    # Display comparison table for KEY_AMOUNTS
    display_comparison_table(max_fee, half_amount, selected_amount)

def display_comparison_table(max_fee, half_amount, selected_amount):
    """Display comparison table for key transfer amounts"""
    
    rows_html = ""
    
    for i, amt in enumerate(KEY_AMOUNTS):
        lin_fee = linear_fee(amt, max_fee, half_amount)
        prog_fee = progressive_fee(amt, max_fee, half_amount)
        reg_fee = regressive_fee(amt, max_fee, half_amount)
        
        lin_pct = (lin_fee / amt) * 100 if amt > 0 else 0
        prog_pct = (prog_fee / amt) * 100 if amt > 0 else 0
        reg_pct = (reg_fee / amt) * 100 if amt > 0 else 0
        
        # Highlight selected row
        row_style = 'background-color: #fff3cd; font-weight: bold;' if amt == selected_amount else ''
        
        # Convert to dollars for display
        amt_dollars = amt / 10**6
        lin_fee_dollars = lin_fee / 10**6
        prog_fee_dollars = prog_fee / 10**6
        reg_fee_dollars = reg_fee / 10**6
        
        rows_html += f"""
        <tr style="{row_style}">
            <td style="padding: 10px; border: 1px solid #dee2e6; text-align: center;">
                <span style="display: inline-block; width: 12px; height: 12px; background-color: {KEY_AMOUNT_COLORS[i]}; border-radius: 50%; margin-right: 8px;"></span>
                <strong>{KEY_AMOUNT_LABELS[i]}</strong>
            </td>
            <td style="padding: 10px; border: 1px solid #dee2e6; background-color: rgba(46, 134, 171, 0.1);">${lin_fee_dollars:,.2f}</td>
            <td style="padding: 10px; border: 1px solid #dee2e6; background-color: rgba(46, 134, 171, 0.05);">{lin_pct:.3f}%</td>
            <td style="padding: 10px; border: 1px solid #dee2e6; background-color: rgba(162, 59, 114, 0.1);">${prog_fee_dollars:,.2f}</td>
            <td style="padding: 10px; border: 1px solid #dee2e6; background-color: rgba(162, 59, 114, 0.05);">{prog_pct:.3f}%</td>
            <td style="padding: 10px; border: 1px solid #dee2e6; background-color: rgba(241, 143, 1, 0.1);">${reg_fee_dollars:,.2f}</td>
            <td style="padding: 10px; border: 1px solid #dee2e6; background-color: rgba(241, 143, 1, 0.05);">{reg_pct:.3f}%</td>
        </tr>
        """
    
    html = f"""
    <div style="margin-top: 20px; padding: 15px; background-color: #f8f9fa; border-radius: 8px; border: 1px solid #dee2e6;">
        <h3 style="margin-top: 0; color: #212529;">📊 Key Transfer Amounts Comparison</h3>
        <table style="width: 100%; border-collapse: collapse; margin-top: 10px;">
            <thead>
                <tr style="background-color: #e9ecef; text-align: center;">
                    <th style="padding: 10px; border: 1px solid #dee2e6;">Transfer Amount</th>
                    <th colspan="2" style="padding: 10px; border: 1px solid #dee2e6; background-color: rgba(46, 134, 171, 0.2);">Linear</th>
                    <th colspan="2" style="padding: 10px; border: 1px solid #dee2e6; background-color: rgba(162, 59, 114, 0.2);">Progressive</th>
                    <th colspan="2" style="padding: 10px; border: 1px solid #dee2e6; background-color: rgba(241, 143, 1, 0.2);">Regressive</th>
                </tr>
                <tr style="background-color: #f8f9fa; text-align: center; font-size: 0.9em;">
                    <th style="padding: 8px; border: 1px solid #dee2e6;"></th>
                    <th style="padding: 8px; border: 1px solid #dee2e6;">Fee (USD)</th>
                    <th style="padding: 8px; border: 1px solid #dee2e6;">%</th>
                    <th style="padding: 8px; border: 1px solid #dee2e6;">Fee (USD)</th>
                    <th style="padding: 8px; border: 1px solid #dee2e6;">%</th>
                    <th style="padding: 8px; border: 1px solid #dee2e6;">Fee (USD)</th>
                    <th style="padding: 8px; border: 1px solid #dee2e6;">%</th>
                </tr>
            </thead>
            <tbody>
                {rows_html}
            </tbody>
        </table>
        <p style="margin-top: 15px; margin-bottom: 0; color: #6c757d; font-size: 0.9em;">
            <strong>Parameters:</strong> maxFee = ${max_fee/10**6:,.2f} | halfAmount = ${half_amount/10**6:,.2f}
        </p>
        <p style="margin-top: 5px; margin-bottom: 0; color: #6c757d; font-size: 0.85em;">
            ⭐ <strong>Highlighted row</strong> indicates the currently selected transfer amount | 💡 <strong>1 USDC = 10<sup>6</sup> tokens</strong>
        </p>
    </div>
    """
    display(HTML(html))

### 🎛️ Interactive Controls

Adjust these sliders to tune **maxFee** and **halfAmount**, and select a transfer amount to analyze:

In [ ]:
# Create slider layout
slider_layout = Layout(width='600px')

# Interactive widget
interact(
    plot_fees,
    max_fee=IntSlider(
        value=1 * 10**6,
        min=100000,
        max=10 * 10**6,
        step=100000,
        description='Max Fee (USD):',
        style={'description_width': '150px'},
        layout=slider_layout
    ),
    half_amount=IntSlider(
        value=10 * 10**6,
        min=1 * 10**6,
        max=100 * 10**6,
        step=1 * 10**6,
        description='Half Amount (USD):',
        style={'description_width': '150px'},
        layout=slider_layout
    ),
    selected_amount=Dropdown(
        options=[
            ('$100', 100 * 10**6),
            ('$1,000', 1000 * 10**6),
            ('$10,000', 10000 * 10**6),
            ('$100,000', 100000 * 10**6),
            ('Custom: $5,000', 5000 * 10**6),
            ('Custom: $25,000', 25000 * 10**6),
            ('Custom: $50,000', 50000 * 10**6)
        ],
        value=10000 * 10**6,
        description='Selected Amount:',
        style={'description_width': '150px'},
        layout=slider_layout
    )
);

## 📊 Formula Reference

### Linear Fee
```
fee = min(maxFee, (amount × maxFee) / (2 × halfAmount))
```
- Simple linear growth until reaching cap
- Most predictable and straightforward

### Progressive Fee
```
fee = (maxFee × amount²) / (halfAmount² + amount²)
```
- Fee percentage increases up to halfAmount, then decreases
- Encourages mid-sized transfers

### Regressive Fee
```
fee = (maxFee × amount) / (halfAmount + amount)
```
- Fee percentage continuously decreases
- Most favorable for large transfers ("whale-friendly")

---

**Key Insight:** All three curves intersect at `(halfAmount, maxFee/2)` ✨

**Key Amounts:** The visualization highlights 4 key transfer amounts ($100, $1K, $10K, $100K) with colored markers to help compare fee structures across common transaction sizes.